Generates one Panel with Offset, and one panel without offset from tracking of rotation.
Tested for timestamp 4020 and 4025.
Results follow closely the panel and the height is adjusted accordingly.


In [1]:
import sys

sys.path.insert(0, '../bifacial_radiance')

try:
    from bifacial_radiance import *
    print " Successful import of bifacial_radiance version . "
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.

 Successful import of bifacial_radiance version . 


In [4]:
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Octtest'

demo = RadianceObj('simple_panel',path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(0.62) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
try:
    epwfile = demo.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
except:
    pass

metdata = demo.readEPW(epwfile) # read in the EPW weather data from above
#metdata = demo.readTMY() # select a TMY file using graphical picker
# Now we either choose a single time point, or use cumulativesky for the entire year. 
demo.gendaylit(metdata,4025)  # Noon, June 17th


timeindex = 4020 # Noon, June 17th. 
simulationname = 'NEW_tracker'+str(timeindex)

# MakeModule Parameters
module_type='my_little_pony_panel'
module_length = 1.996  # 2-up portrait Longi with 15cm additional gap
module_width = 0.991
orientation='portrait' 
tilt = 10

# TorqueTube Parameters
torqueTube = True
numpanels = 2
gap = 0.15
tubetype = 'Hex'
tubeZgap = 0.05
diameter = 0.15
torqueTubeMaterial = 'Metal_Grey'       # IT's NOT GRAY, IT's GREY
axisofrotationTorqueTube = False

# Tracking Angle Calculation Parameters
hub_height = 2.35   
axis_azimuth = 180.0
roundTrackerAngleBool = False 
angledelta = 5
limit_angle = 60
backtrack = True
# axisofrotationTorqueTube=False, diameter=0.1, tubeZgap=0.1
# axisofrotationTorqueTube=axisofrotationTorqueTube, diameter=diameter, tubeZgap=tubeZgap

# SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_he
nMods = 1
nRows=1
sensorsy=200
psx = 0.05

#We have a 2-up configuration in portrait. This section helps sort what is the COLLECTOR WIDTH to use.
if orientation == 'portrait':
                slope=module_length
elif orientation == 'landscape':  
                slope=module_width
collectorwidth = slope*numpanels+gap*(numpanels-1) # If it's 1UP, the gap value will not matter. This is a safety.


demo.makeModule(name=module_type,x=module_width,y=module_length,bifi=1,orientation='portrait', 
       torquetube=torqueTube, diameter = diameter, tubetype = tubetype, material = torqueTubeMaterial, tubeZgap=tubeZgap, numpanels=numpanels, psx=psx, axisofrotationTorqueTube=axisofrotationTorqueTube)


tracker_theta, tracker_height, azimuth_ang = demo._getTrackingGeometryTimeIndex(metdata, timeindex=timeindex, angledelta = angledelta, 
        roundTrackerAngleBool = roundTrackerAngleBool, axis_azimuth = axis_azimuth, 
        limit_angle = limit_angle, backtrack = backtrack, gcr = gcr, hubheight = hub_height, module_height = collectorwidth, axisofrotationTorqueTube=axisofrotationTorqueTube, diameter=diameter, tubeZgap=tubeZgap)


sceneDict = {'tilt':tracker_theta,'pitch': np.round(collectorwidth / gcr,3),'height':tracker_height,'orientation':orientation,'azimuth':azimuth_ang, 'module_type':module_type}  

scene = demo.makeScene(moduletype=module_type,sceneDict=sceneDict, nMods = nMods, nRows = nRows, sensorsy=sensorsy, psx=psx, axisofrotationTorqueTube=axisofrotationTorqueTube, diameter=diameter, tubeZgap=tubeZgap) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.
analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance
analysis.analysis(octfile, "Regular", scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )

# PPART 2
axisofrotationTorqueTube = True
module_type='my_little_pony_panel_OFFSET'

demo.makeModule(name=module_type,x=module_width,y=module_length,bifi=1,orientation='portrait', 
       torquetube=torqueTube, diameter = diameter, tubetype = tubetype, material = torqueTubeMaterial, tubeZgap=tubeZgap, numpanels=numpanels, psx=psx, axisofrotationTorqueTube=axisofrotationTorqueTube)

tracker_theta, tracker_height, azimuth_ang = demo._getTrackingGeometryTimeIndex(metdata, timeindex=timeindex, angledelta = angledelta, 
        roundTrackerAngleBool = roundTrackerAngleBool, axis_azimuth = axis_azimuth, 
        limit_angle = limit_angle, backtrack = backtrack, gcr = gcr, hubheight = hub_height, module_height = collectorwidth, axisofrotationTorqueTube=axisofrotationTorqueTube, diameter=diameter, tubeZgap=tubeZgap)

#We have a 2-up configuration in portrait. This section helps sort what is the COLLECTOR WIDTH to use.
if orientation == 'portrait':
                slope=module_length
elif orientation == 'landscape':  
                slope=module_width
collectorwidth = slope*numpanels+gap*(numpanels-1) # If it's 1UP, the gap value will not matter. This is a safety.

sceneDict = {'tilt':tracker_theta,'pitch': np.round(collectorwidth / gcr,3),'height':tracker_height,'orientation':orientation,'azimuth':azimuth_ang, 'module_type':module_type}  

scene = demo.makeScene(moduletype=module_type,sceneDict=sceneDict, nMods = nMods, nRows = nRows,  sensorsy=sensorsy, psx=psx, axisofrotationTorqueTube=axisofrotationTorqueTube, diameter=diameter, tubeZgap=tubeZgap) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.
analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance
analysis.analysis(octfile, "Offset", scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )


path = C:\Users\sayala\Documents\RadianceScenes\Octtest
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
('\nModule Name:', 'my_little_pony_panel')
REWRITING pre-existing module file. 
Module my_little_pony_panel successfully created
For this timestamp, panels are facing West
Tracker theta has been calculated to 4.722, no rounding performed.
Module clearance height has been calculated to 2.179, for this tracker theta.
Created simple_panel.oct
Linescan in process: Regular_Front
Linescan in process: Regular_Back
Saved: results\irr_Regular.csv
Annual bifacial ratio average:  0.536
('\nModule Name:', 'my_little_pony_panel_OFFSET')
REWRITING pre-existing module file. 
Module my_little_pony_panel_OFFSET successfully created
For this timestamp, panels are facing West
Tracker theta has been calculated to 4.722, no rounding performed.
Considering offset from axis of rotation of torque tube. Height without shift: 2.179
Module clearance height has been calculated to 2.379, fo

In [5]:
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.


Created simple_panel.oct
